# Call Reports Analysis

##### Housekeeping and loading data

In [1]:
# Load packages
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels import PanelOLS
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def binned_scatter(x, y, q, marker='o'):
    # Create quantile bins using qcut
    x_binned, bin_edges = pd.qcut(x, q=q, retbins=True, duplicates='drop')
    bin_centers = []
    binned_means = []
    binned_median = []
    binned_min = []
    binned_max = []
    
    # For each unique bin, compute center, mean, min and max of y
    for interval in x_binned.unique():
        x_in_bin = x[x_binned == interval]
        y_in_bin = y[x_binned == interval]
        bin_center = x_in_bin.mean()
        mean_val = y_in_bin.mean()
        median_val = y_in_bin.median()
        min_val = y_in_bin.quantile(0.75)
        max_val = y_in_bin.quantile(0.25)
        bin_centers.append(bin_center)
        binned_means.append(mean_val)
        binned_median.append(median_val)
        binned_min.append(min_val)
        binned_max.append(max_val)
    
    # Sort the results based on bin centers
    sorted_indices = np.argsort(bin_centers)
    bin_centers = np.array(bin_centers)[sorted_indices]
    binned_means = np.array(binned_means)[sorted_indices]
    binned_median = np.array(binned_median)[sorted_indices]
    binned_min = np.array(binned_min)[sorted_indices]
    binned_max = np.array(binned_max)[sorted_indices]
    
    # Plot the scatter for the mean with full opacity
    #plt.scatter(bin_centers, binned_max, label='75%', marker='o',
    #            alpha=0.1, s=50, color='grey')
    plt.scatter(bin_centers, binned_means, label='mean', marker=marker,
                alpha=1, s=50)
    #plt.scatter(bin_centers, binned_median, label='median', marker='o',
    #            alpha=0.1, s=50, color='green')
    #plt.scatter(bin_centers, binned_min, label='25%', marker='o',
    #            alpha=0.1, s=50, color='grey')
    
    plt.xlabel(x.name)
    plt.ylabel(y.name)
    plt.grid(True, linestyle='--', alpha=0.5, linewidth=0.5, color='lightgrey')

In [3]:
def get_shared_bins(x, y, q):
    """Compute shared quantile bins for both x and y"""
    combined = pd.concat([x, y])
    _, bin_edges = pd.qcut(combined, q=q, retbins=True, duplicates='drop')
    return bin_edges

# For each plot, reuse these components
def add_45_degree_line():
    ax = plt.gca()
    lims = [min(ax.get_xlim()[0], ax.get_ylim()[0]), 
            max(ax.get_xlim()[1], ax.get_ylim()[1])]
    plt.plot(lims, lims, 'k--', alpha=0.7, lw=1, label='45-degree line')
    plt.xlim(lims)
    plt.ylim(lims)

In [4]:
path = 'C:/Users/angel/Documents/Economics/Research/Banking Project/data/clean'
path_output = 'C:/Users/angel/Documents/Economics/Research/Banking Project/data/output'

# set colorblind theme for plots:
sns.set_theme(context='notebook', style="ticks", palette='colorblind')
sns.set_color_codes(palette='colorblind')

# Set path to be the directory:
os.chdir(path)

In [5]:
# read just the first row in the 'call_reports.csv' file:
sample = pd.read_csv('call_reports.csv', nrows=1)

# list the columns that have '_x' and '_y' in them:
cols_x = [col for col in sample.columns if '_x' in col]
cols_y = [col for col in sample.columns if '_y' in col]

problem_cols = cols_x + cols_y
problem_cols

['RCFD1754_x',
 'RCFD1773_x',
 'RCFD2930_x',
 'RCON1754_x',
 'RCON2930_x',
 'Unnamed: 79_x',
 'Unnamed: 241_x',
 'Unnamed: 88_x',
 'RCFD2930_y',
 'RCON2930_y',
 'RCFD1754_y',
 'RCFD1773_y',
 'RCON1754_y',
 'Unnamed: 79_y',
 'Unnamed: 241_y',
 'Unnamed: 88_y']

In [6]:
# define maturity variables:
loans_mat_vars = [
                'RCONA564', 'RCONA565', 'RCONA566', 'RCONA567', 'RCONA568', 'RCONA569',     # used
                #'RCFDA564', 'RCFDA565', 'RCFDA566', 'RCFDA567', 'RCFDA568', 'RCFDA569',     # to be tested
                # ------------------------------------------------------------------------------------------------
                'RCFDA570', 'RCFDA571', 'RCFDA572', 'RCFDA573', 'RCFDA574', 'RCFDA575',     # used 
                #'RCONA570', 'RCONA571', 'RCONA572', 'RCONA573', 'RCONA574', 'RCONA575',     # to be tested  
                ]

securities_mat_vars = [
             # --------------------------------------  Treasuries  --------------------------------------
                'RCFDA549', 'RCFDA550', 'RCFDA551', 'RCFDA552', 'RCFDA553', 'RCFDA554',     # used
                'RCONA549', 'RCONA550', 'RCONA551', 'RCONA552', 'RCONA553', 'RCONA554',     # to be tested
            # --------------------------------------  MBS  --------------------------------------
                'RCFDA555', 'RCFDA556', 'RCFDA557', 'RCFDA558', 'RCFDA559', 'RCFDA560',     # used
                'RCONA555', 'RCONA556', 'RCONA557', 'RCONA558', 'RCONA559', 'RCONA560',     # to be tested
                ]


# define the list of variables that will be used
vars = [
             # ------------------------------------------------------------------------------------------------ 
             'Date', 'IDRSSD', 'Financial Institution Name',            # Identifier Variables
             # ------------------------------------------------------------------------------------------------
            'RCON2170', 'RCFD2170',                                     # Total Assets
            'RCON3368', 'RCFD3368',                                     # QA Total Assets
             # ------------------------------------------------------------------------------------------------
             'RCON2122', 'RCFD2122',                                    # Total Loans
             # ------------------------------------------------------------------------------------------------
             'RCON2200',                                                # Total Deposits
             'RCON2215',                                                # Transaction Deposits
             'RCON6648',                                                # 0-100k
             'RCON2604',                                                # 100k+ (old)
             'RCONJ473',                                                # 100-250k (new)
             'RCONJ474',                                                # 250k+ (new)
             # ------------------------------------------------------------------------------------------------
             'RCONF045', 'RCONF046', 'RCONF047', 
             'RCONF048', 'RCONF049', 'RCONF050',
             'RCONF051', 'RCONF052', 'RCON3645',                        # Uninsured Deposits
             # ------------------------------------------------------------------------------------------------
             'RCON3210', 'RCFD3210',                                    # Total Equity Capital
             'RCONB530', 'RCFDB530',                                    # AOCI
             'RCON2948', 'RCFD2948',                                    # Total Liabilities
             # ------------------------------------------------------------------------------------------------
             'RCON1754', 'RCFD1754',                                    # HTM Securities Ammortized Cost
             'RCFD1754_x', 'RCFD1754_y', 'RCON1754_x', 'RCON1754_y',
             # ------------------------------------------------------------------------------------------------
             'RCON1771', 'RCFD1771',                                    # HTM Securities Fair Value
             # ------------------------------------------------------------------------------------------------
             'RCON1772', 'RCFD1772',                                    # AFS Securities Ammortized Cost
             # ------------------------------------------------------------------------------------------------
             'RCONJJ34', 'RCFDJJ34',                                    
             'RCONJA22', 'RCFDJA22',                                    # Booked Securities
             # ------------------------------------------------------------------------------------------------
             'RCFD1773_x', 'RCFD1773_y', 'RCON1773',                    # AFS Securities Fair Value
             # ------------------------------------------------------------------------------------------------
             'RCONB987',                                                # FF sold in domestic offices
             'RCONB989', 'RCFDB989',                                    # Resell agreements                                  
             # ------------------------------------------------------------------------------------------------
             'RCFD0081', 'RCON0081',                                    # Cash and Balances Due from Depository Institutions
             'RCFD0071', 'RCON0071',
             'RCFD0010', 'RCON0010',
             # ------------------------------------------------------------------------------------------------
             'RIAD4508',                                                 # Interest Expenses
             'RIADA517', 'RIADA518', 'RIAD0093',
             'RIADHK03', 'RIADHK04', 'RIAD4073', 'RIAD4200', 'RIAD4185', 
             'RIAD4180', 'RIAD4172', 'RIAD4107', 'RIAD4340', 'RIAD4093'
             ] 

# create a list putting together 'vars', 'loans_mat_vars', and 'securities_mat_vars':
all_vars = vars + loans_mat_vars + securities_mat_vars

In [7]:
# Read file that contains only the variables of interest, specify that the column 'Date' is a date:
main = pd.read_csv('call_reports.csv', parse_dates = ['Date'], usecols=all_vars)
main['Year'] = main['Date'].dt.year

In [8]:
main[['IDRSSD', 'RCONA567']].head(10)

,IDRSSD,RCONA567
0,242,991
1,279,1266
2,354,0
3,439,4919
4,457,222
5,505,702
6,1146,3275
7,1155,5777
8,1351,263
9,1454,3979


##### Checking important variables

In [10]:
print('-------------------- TOTAL LIABILITIES --------------------')
# print the obs in which 'RCON2948' is reported and 'RCFD2948' is not:
print('For how many non-null obs. RCON2948 is reported and RCFD2948 is not:',
      len(main[main['RCON2948'].notnull() & main['RCFD2948'].isnull()]))
# print the obs in which 'RCON2948' is not reported and 'RCFD2948' is:
print('For how many non-null obs. RCFD2948 is reported and RCON2948 is not:',
      len(main[main['RCFD2948'].notnull() & main['RCON2948'].isnull()]))
# print the obs in which 'RCON2948' is reported and 'RCFD2948' is reported:
print('For how many non-null obs. RCON2948 and RCFD2948 are both reported:',
      len(main[main['RCON2948'].notnull() & main['RCFD2948'].notnull()]))
# for the obs in which both are reported, print the positive difference:
print('For how many non-null obs. RCON2948 and RCFD2948 are both reported and the difference is positive:',
      len(main[(main['RCON2948'].notnull() & main['RCFD2948'].notnull()) &
               (main['RCON2948'] - main['RCFD2948']!=0)]))

-------------------- TOTAL LIABILITIES --------------------
For how many non-null obs. RCON2948 is reported and RCFD2948 is not: 639209
For how many non-null obs. RCFD2948 is reported and RCON2948 is not: 4933
For how many non-null obs. RCON2948 and RCFD2948 are both reported: 4820
For how many non-null obs. RCON2948 and RCFD2948 are both reported and the difference is positive: 2688


In [ ]:
print(' ---------------------------------------------- AMMORTIZED COST SECURITIES ----------------------------------------------')

# print the latest 'Date' for which either 'RCON1754_x' or 'RCON1754_y' is reported:
print('Min Date in which RCFD1754_x or RCFD1754_y are not null:', 
      main[main['RCFD1754_x'].notnull() | main['RCFD1754_y'].notnull()]['Date'].min())
print('Max Date in which RCFD1754_x or RCFD1754_y are not null:',
      main[main['RCFD1754_x'].notnull() | main['RCFD1754_y'].notnull()]['Date'].max())

print('Min Date in which RCFD1754 is not null:', main[main['RCFD1754'].notnull()]['Date'].min())
print('Max Date in which RCFD1754 is not null:', main[main['RCFD1754'].notnull()]['Date'].max())

print('------------------------------------------------------------------------------------------------')

# print the latest 'Date' for which either 'RCON1754_x' or 'RCON1754_y' is reported:
print('Min Date in which RCON1754_x or RCON1754_y are not null:',
      main[main['RCON1754_x'].notnull() | main['RCON1754_y'].notnull()]['Date'].min())
print('Max Date in which RCON1754_x or RCON1754_y are not null:',
      main[main['RCON1754_x'].notnull() | main['RCON1754_y'].notnull()]['Date'].max())

print('Min Date in which RCON1754 is not null:',main[main['RCON1754'].notnull()]['Date'].min())
print('Max Date in which RCON1754 is not null:', main[main['RCON1754'].notnull()]['Date'].max())

print('------------------------------------------------------------------------------------------------')

print('The number of rows in which RCON1754_x and RCON1754_y are both reported and different is:',
    len(main[main['RCON1754_x'].notnull() & main['RCON1754_y'].notnull() & (main['RCON1754_x']-main['RCON1754_y'] != 0)]))

print('The number of rows in which RCFD1754_x and RCFD1754_y are both reported and different is:',
    len(main[main['RCFD1754_x'].notnull() & main['RCFD1754_y'].notnull() & (main['RCFD1754_x']-main['RCFD1754_y'] != 0)]))

In [ ]:
print(' ---------------------------------------------- FAIR VALUE SECURITIES ----------------------------------------------')

# print the latest 'Date' for which either 'RCON1754_x' or 'RCON1754_y' is reported:
print('Min Date in which RCFD1773_x and RCFD1773_y are not null:', 
      main[main['RCFD1773_x'].notnull() & main['RCFD1773_y'].notnull()]['Date'].min())
print('Max Date in which RCFD1773_x and RCFD1773_y are not null:',
      main[main['RCFD1773_x'].notnull() & main['RCFD1773_y'].notnull()]['Date'].max())

print('For how many non-null obs. the difference between RCFD1773_x and RCFD1773_y is not zero:',
      len(main[main['RCFD1773_x'].notnull() & 
     main['RCFD1773_y'].notnull() & 
     (main['RCFD1773_x']-main['RCFD1773_y'] != 0)]))
print('We can treat _x and _y as the same variable, since the difference is zero for all non-null obs.')
print('------------------------------------------------------------------------------------------------')
# print the amount of obs for which RCFD1773_x and RCON1773 are both reported and different:
print('For how many non-null obs. RCFD1773_x and RCON1773 are both reported and different:',
      len(main[main['RCFD1773_x'].notnull() & 
     main['RCON1773'].notnull() & 
     (main['RCFD1773_x']-main['RCON1773'] != 0)]))
# print the amount of obs for which RCFD1773_x and RCON1773 are both reported and are the same:
print('For how many non-null obs. RCFD1773_x and RCON1773 are both reported and the same:',
      len(main[main['RCFD1773_x'].notnull() & 
     main['RCON1773'].notnull() & 
     (main['RCFD1773_x']-main['RCON1773'] == 0)]))

# print the amount of obs for which RCFD1773_x is reported and RCON1773 is not:
print('For how many non-null obs. RCFD1773_x is reported and RCON1773 is not:',
      len(main[main['RCFD1773_x'].notnull() & main['RCON1773'].isnull()]))

# print the amount of obs for which RCFD1773_x is not reported and RCON1773 is:
print('For how many non-null obs. RCFD1773_x is not reported and RCON1773 is:',
      len(main[main['RCFD1773_x'].isnull() & main['RCON1773'].notnull()]))


print('------------------------------------------------------------------------------------------------')

In [ ]:

print(' ---------------------------------------------- Cash ----------------------------------------------')


# print the obs in which 'RCON0010' is reported and 'RCFD0010' is not:
print('For how many non-null obs. RCON0010 is reported and RCFD0010 is not:',
      len(main[main['RCON0010'].notnull() & main['RCFD0010'].isnull()]))
# print the obs in which 'RCON0010' is not reported and 'RCFD0010' is:
print('For how many non-null obs. RCON0010 is not reported and RCFD0010 is:',
      len(main[main['RCON0010'].isnull() & main['RCFD0010'].notnull()]))
# print the obs in which 'RCON0010' and 'RCFD0010' are both reported:
print('For how many non-null obs. RCON0010 and RCFD0010 are both reported:',
      len(main[main['RCON0010'].notnull() & main['RCFD0010'].notnull()]))
# for the observations where RCON0010 and RCFD0010 are both reported, print the maximum difference between them:
print('For how many non-null obs. the difference between RCON0010 and RCFD0010 is not zero:',
      len(main[main['RCON0010'].notnull() & main['RCFD0010'].notnull() &
     (main['RCON0010']-main['RCFD0010'] == 0)]))

print('------------------------------------------------------------------------------------------------')
# print the obs in which 'RCON0071' is reported and 'RCFD0071' is not:
print('For how many non-null obs. RCON0071 is reported and RCFD0071 is not:',
      len(main[main['RCON0071'].notnull() & main['RCFD0071'].isnull()]))
# print the obs in which 'RCON0071' is not reported and 'RCFD0071' is:
print('For how many non-null obs. RCON0071 is not reported and RCFD0071 is:',
      len(main[main['RCON0071'].isnull() & main['RCFD0071'].notnull()]))
# print the obs in which 'RCON0071' and 'RCFD0071' are both reported:
print('For how many non-null obs. RCON0071 and RCFD0071 are both reported:',
      len(main[main['RCON0071'].notnull() & main['RCFD0071'].notnull()]))
# for the observations where RCON0071 and RCFD0071 are both reported, print the maximum difference between them:
print('For how many non-null obs. the difference between RCON0071 and RCFD0071 is not zero:',
      len(main[main['RCON0071'].notnull() & main['RCFD0071'].notnull() &
     (main['RCON0071']-main['RCFD0071'] == 0)]))
print('------------------------------------------------------------------------------------------------')
# print the obs in which 'RCON0081' is reported and 'RCFD0081' is not:
print('For how many non-null obs. RCON0081 is reported and RCFD0081 is not:',
      len(main[main['RCON0081'].notnull() & main['RCFD0081'].isnull()]))
# print the obs in which 'RCON0081' is not reported and 'RCFD0081' is:
print('For how many non-null obs. RCON0081 is not reported and RCFD0081 is:',
      len(main[main['RCON0081'].isnull() & main['RCFD0081'].notnull()]))
# print the obs in which 'RCON0081' and 'RCFD0081' are both reported:
print('For how many non-null obs. RCON0081 and RCFD0081 are both reported:',
      len(main[main['RCON0081'].notnull() & main['RCFD0081'].notnull()]))
# for the observations where RCON0081 and RCFD0081 are both reported, print the maximum difference between them:
print('For how many non-null obs. the difference between RCON0081 and RCFD0081 is not zero:',
      len(main[main['RCON0081'].notnull() & main['RCFD0081'].notnull() &
     (main['RCON0081']-main['RCFD0081'] == 0)]))

In [ ]:
print(' --------------------------- Maturity for Securities ----------------------------------------')
# print the last 3 digits from 'securities_mat_vars':
last_digits = sorted(list(set([var[-3:] for var in securities_mat_vars])))

for x in last_digits:
    print(' --------------------------------------------------------------------------------------------')
    print('For x =', x)
    print(f'Entries in which both are null:', len(main[(main[f'RCONA{x}'].isnull()) & (main[f'RCFDA{x}'].isnull())]))
    print(f'Entries in which only RCFDA{x} is NOT null:', len(main[(main[f'RCONA{x}'].isnull()) & (main[f'RCFDA{x}'].notnull())]))
    print(f'Entries in which only RCONA{x} is NOT null:', len(main[(main[f'RCONA{x}'].notnull()) & (main[f'RCFDA{x}'].isnull())]))
    print(f'Entries in which none is null:', len(main[(main[f'RCONA{x}'].notnull()) & (main[f'RCFDA{x}'].notnull())]))

In [ ]:
print(' --------------------------- Maturity for Loans ----------------------------------------')
print('-------------------------------------- Less than 3m --------------------------------------')
print(f'Entries in which both are null:', len(main[(main['RCONA564'].isnull()) & (main['RCFDA570'].isnull())]))
print(f'Entries in which only RCFDA570 is NOT null:', len(main[(main['RCONA564'].isnull()) & (main['RCFDA570'].notnull())]))
print(f'Entries in which only RCONA564 is NOT null:', len(main[(main['RCONA564'].notnull()) & (main['RCFDA570'].isnull())]))
print(f'Entries in which none is null:', len(main[(main['RCONA564'].notnull()) & (main['RCFDA570'].notnull())]))
print('-------------------------------------- Between 3-12m --------------------------------------')
print(f'Entries in which both are null:', len(main[(main['RCONA565'].isnull()) & (main['RCFDA571'].isnull())]))
print(f'Entries in which only RCFDA571 is NOT null:', len(main[(main['RCONA565'].isnull()) & (main['RCFDA571'].notnull())]))
print(f'Entries in which only RCONA565 is NOT null:', len(main[(main['RCONA565'].notnull()) & (main['RCFDA571'].isnull())]))
print(f'Entries in which none is null:', len(main[(main['RCONA565'].notnull()) & (main['RCFDA571'].notnull())]))
print('-------------------------------------- Between 12-36m --------------------------------------')
print(f'Entries in which both are null:', len(main[(main['RCONA566'].isnull()) & (main['RCFDA572'].isnull())]))
print(f'Entries in which only RCFDA572 is NOT null:', len(main[(main['RCONA566'].isnull()) & (main['RCFDA572'].notnull())]))
print(f'Entries in which only RCONA566 is NOT null:', len(main[(main['RCONA566'].notnull()) & (main['RCFDA572'].isnull())]))
print(f'Entries in which none is null:', len(main[(main['RCONA566'].notnull()) & (main['RCFDA572'].notnull())]))
print('-------------------------------------- Between 36-60m --------------------------------------')
print(f'Entries in which both are null:', len(main[(main['RCONA567'].isnull()) & (main['RCFDA573'].isnull())]))
print(f'Entries in which only RCFDA573 is NOT null:', len(main[(main['RCONA567'].isnull()) & (main['RCFDA573'].notnull())]))
print(f'Entries in which only RCONA567 is NOT null:', len(main[(main['RCONA567'].notnull()) & (main['RCFDA573'].isnull())]))
print(f'Entries in which none is null:', len(main[(main['RCONA567'].notnull()) & (main['RCFDA573'].notnull())]))
print('-------------------------------------- Between 60-180m --------------------------------------')
print(f'Entries in which both are null:', len(main[(main['RCONA568'].isnull()) & (main['RCFDA574'].isnull())]))
print(f'Entries in which only RCFDA574 is NOT null:', len(main[(main['RCONA568'].isnull()) & (main['RCFDA574'].notnull())]))
print(f'Entries in which only RCONA568 is NOT null:', len(main[(main['RCONA568'].notnull()) & (main['RCFDA574'].isnull())]))
print(f'Entries in which none is null:', len(main[(main['RCONA568'].notnull()) & (main['RCFDA574'].notnull())]))
print('-------------------------------------- More than 180m --------------------------------------')
print(f'Entries in which both are null:', len(main[(main['RCONA569'].isnull()) & (main['RCFDA575'].isnull())]))
print(f'Entries in which only RCFDA575 is NOT null:', len(main[(main['RCONA569'].isnull()) & (main['RCFDA575'].notnull())]))
print(f'Entries in which only RCONA569 is NOT null:', len(main[(main['RCONA569'].notnull()) & (main['RCFDA575'].isnull())]))
print(f'Entries in which none is null:', len(main[(main['RCONA569'].notnull()) & (main['RCFDA575'].notnull())]))

##### Basic Definitions

###### Balance sheet definitions

In [ ]:
main['RCON1754_right'] = main['RCON1754_x'].combine_first(main['RCON1754'])
main['RCFD1754_right'] = main['RCFD1754_x'].combine_first(main['RCFD1754'])

# Create the new column '1754_right' and initialize with NaN
main['1754_right'] = np.nan

# Case 1: Both columns are not null
mask_both_notnull = main['RCFD1754_right'].notna() & main['RCON1754_right'].notna()
main.loc[mask_both_notnull, '1754_right'] = main.loc[mask_both_notnull, ['RCFD1754_right', 'RCON1754_right']].min(axis=1)

# Case 2: Only one column is not null
mask_col1_notnull = main['RCFD1754_right'].notna() & main['RCON1754_right'].isna()
main.loc[mask_col1_notnull, '1754_right'] = main.loc[mask_col1_notnull, 'RCFD1754_right']

mask_col2_notnull = main['RCON1754_right'].notna() & main['RCFD1754_right'].isna()
main.loc[mask_col2_notnull, '1754_right'] = main.loc[mask_col2_notnull, 'RCON1754_right']

# Case 3: Both columns are null (already handled by initialization to NaN)


In [ ]:
print(main['RCON1754_right'].describe())
print(main['RCFD1754_right'].describe())
print(main['1754_right'].describe())

In [ ]:
main[main['RCON1754_right'].notnull() & 
     main['RCFD1754_right'].notnull() & 
     (main['RCON1754_right']-main['RCFD1754_right'] != 0)][['Date', 'RCON1754_right', 'RCFD1754_right', '1754_right']]

In [ ]:
# create 'Securities AC' summing RCON1754 and RCON1772. If both are NaN, fill with NaN. If only one is available, use that one. If both 
# are available sum them:
main['Securities AC'] = main[['1754_right', 'RCON1772']].apply(
    lambda row: np.nan if pd.isna(row['1754_right']) and pd.isna(row['RCON1772']) else (row.fillna(0).sum()),
    axis=1
)
main['Securities AC'] = main['Securities AC'].mask(main['Securities AC'] <= 0, np.nan)
print(main['Securities AC'].describe())

In [ ]:
main['1773_right'] = main['RCON1773'].combine_first(main['RCFD1773_x'])
# mask the <= 0 values with NaN:
main['1773_right'] = main['1773_right'].mask(main['1773_right'] <= 0, np.nan)
print(main['1773_right'].describe())

In [ ]:
# Define 'Total Securities' using the sum of '1754_right' and '1773_right' (amortized cost + fair value):
am_cost = True
if am_cost:
    main['Total Securities'] = main['Securities AC']
else:
    main['Total Securities'] = main[['1754_right', '1773_right']].apply(
    lambda row: np.nan if pd.isna(row['1754_right']) and pd.isna(row['1773_right']) else (row.fillna(0).sum()),
    axis=1
    )


main['Total Securities'] = main['Total Securities'].mask(main['Total Securities'] <= 0, np.nan)
print(main['Total Securities'].describe())

In [ ]:
main['Total Loans'] = main['RCON2122'].combine_first(main['RCFD2122'])

print(main['RCON2122'].describe())
print(main['RCFD2122'].describe())
print(main['Total Loans'].describe())

main['Total Loans'] = main['Total Loans'].mask(main['Total Loans'] == 0, np.nan)
print(main['Total Loans'].describe())

In [ ]:
# create 'Cash' summing RCON0020 and RCON0080:
main['Cash'] = (main['RCON0071']+main['RCON0081']).combine_first(main['RCON0010'])
main['Cash'] = main['Cash'].mask(main['Cash'] <= 0, np.nan)
print(main['Cash'].describe())

In [ ]:
# create 'Total Assets' variable. If the bank fills the FFIEC41 (only RCON series available). Large banks may also have the 
# RCFD series available. If both are NaN, fill with NaN. If both are available, use the RCON series. If only RCFD is available,
# use RCFD. 
#main['Total Assets'] = main['RCON2170'].combine_first(main['RCFD2170'])
main['Total Assets'] = main['Cash'] + main['Total Loans'] + main['Total Securities']

print(main['RCON2170'].describe())
print(main['RCFD2170'].describe())
print(main['Total Assets'].describe())

main['Total Assets'] = main['Total Assets'].mask(main['Total Assets'] == 0, np.nan)

In [ ]:
main.loc[:, 'Securities Share'] = main['Total Securities'] / main['Total Assets']

In [ ]:
main['Securities Share'].describe()

In [ ]:
main[main['Securities Share']>1][['Date', 'IDRSSD', 'Financial Institution Name', 'Total Securities', 'Total Assets', 'Securities Share']]

In [ ]:
# create 'Total Deposits'
main.rename(columns = {'RCON2200':'Total Deposits'}, inplace = True)
main['Total Deposits'] = main['Total Deposits'].mask(main['Total Deposits'] == 0, np.nan)
print(main['Total Deposits'].describe())

In [ ]:
# create 'Cash Share' variable:
main.loc[:, 'Cash Share'] = main['Cash'] / main['Total Assets']
print(main['Cash Share'].describe())

In [ ]:
main.loc[:, 'Loans Share'] = main['Total Loans'] / main['Total Assets']
#mask loans share > 1 with NaN:
main['Loans Share'] = main['Loans Share'].mask(main['Loans Share'] > 1, np.nan)
print(main['Loans Share'].describe())

In [ ]:
# Maturity of Treasuries:
main['Treasuries (3M-)'] = main['RCONA549'].combine_first(main['RCFDA549'])
main['Treasuries (3M-1Y)'] = main['RCONA550'].combine_first(main['RCFDA550'])
main['Treasuries (1Y-3Y)'] = main['RCONA551'].combine_first(main['RCFDA551'])
main['Treasuries (3Y-5Y)'] = main['RCONA552'].combine_first(main['RCFDA552'])
main['Treasuries (5Y-15Y)'] = main['RCONA553'].combine_first(main['RCFDA553'])
main['Treasuries (15Y+)'] = main['RCONA554'].combine_first(main['RCFDA554'])

print(main['Treasuries (3M-)'].describe())

# Maturity of MBS:
main['MBS (3M-)'] = main['RCONA555'].combine_first(main['RCFDA555'])
main['MBS (3M-1Y)'] = main['RCONA556'].combine_first(main['RCFDA556'])
main['MBS (1Y-3Y)'] = main['RCONA557'].combine_first(main['RCFDA557'])
main['MBS (3Y-5Y)'] = main['RCONA558'].combine_first(main['RCFDA558'])
main['MBS (5Y-15Y)'] = main['RCONA559'].combine_first(main['RCFDA559'])
main['MBS (15Y+)'] = main['RCONA560'].combine_first(main['RCFDA560'])

print(main['MBS (3M-)'].describe())

# Maturity of overall Securities:
main['Securities (3M-)'] = main['Treasuries (3M-)'] + main['MBS (3M-)']
main['Securities (3M-1Y)'] = main['Treasuries (3M-1Y)'] + main['MBS (3M-1Y)']
main['Securities (1Y-3Y)'] = main['Treasuries (1Y-3Y)'] + main['MBS (1Y-3Y)']
main['Securities (3Y-5Y)'] = main['Treasuries (3Y-5Y)'] + main['MBS (3Y-5Y)']
main['Securities (5Y-15Y)'] = main['Treasuries (5Y-15Y)'] + main['MBS (5Y-15Y)']
main['Securities (15Y+)'] = main['Treasuries (15Y+)'] + main['MBS (15Y+)']

print(main['Securities (3M-)'].describe())

# Maturity of overall Loans:
main['Loans (3M-)'] = main['RCONA564'].combine_first(main['RCFDA570'])
main['Loans (3M-1Y)'] = main['RCONA565'].combine_first(main['RCFDA571'])
main['Loans (1Y-3Y)'] = main['RCONA566'].combine_first(main['RCFDA572'])    
main['Loans (3Y-5Y)'] = main['RCONA567'].combine_first(main['RCFDA573'])
main['Loans (5Y-15Y)'] = main['RCONA568'].combine_first(main['RCFDA574'])
main['Loans (15Y+)'] = main['RCONA569'].combine_first(main['RCFDA575'])

print(main['Loans (3M-)'].describe())


In [ ]:
# create the 'Treasury Maturity' variable:

main['Treasury Maturity'] = ((
                            main['Treasuries (3M-)']*1.5 + 
                            main['Treasuries (3M-1Y)']*7.5 + 
                            main['Treasuries (1Y-3Y)']*24 + 
                            main['Treasuries (3Y-5Y)']*48 + 
                            main['Treasuries (5Y-15Y)']*120 + 
                            main['Treasuries (15Y+)']*180
                            ) / (
                                main['Treasuries (3M-)']        + 
                                main['Treasuries (3M-1Y)']      + 
                                main['Treasuries (1Y-3Y)']      + 
                                main['Treasuries (3Y-5Y)']      + 
                                main['Treasuries (5Y-15Y)']     + 
                                main['Treasuries (15Y+)']
                            )) / 12

print(main['Treasury Maturity'].describe())

# create the 'MBS Maturity' variable:
main['MBS Maturity'] = ((
                            main['MBS (3M-)']*1.5 + 
                            main['MBS (3M-1Y)']*7.5 + 
                            main['MBS (1Y-3Y)']*24 + 
                            main['MBS (3Y-5Y)']*48 + 
                            main['MBS (5Y-15Y)']*120 + 
                            main['MBS (15Y+)']*180
                            ) / (
                                main['MBS (3M-)']        + 
                                main['MBS (3M-1Y)']      + 
                                main['MBS (1Y-3Y)']      + 
                                main['MBS (3Y-5Y)']      + 
                                main['MBS (5Y-15Y)']     + 
                                main['MBS (15Y+)']
                            )) / 12
print(main['MBS Maturity'].describe())

# create the 'Security Maturity' variable:
main['Security Maturity'] = ((
                            main['Securities (3M-)']*1.5 + 
                            main['Securities (3M-1Y)']*7.5 + 
                            main['Securities (1Y-3Y)']*24 + 
                            main['Securities (3Y-5Y)']*48 + 
                            main['Securities (5Y-15Y)']*120 + 
                            main['Securities (15Y+)']*180
                            ) / (
                                main['Securities (3M-)']        + 
                                main['Securities (3M-1Y)']      + 
                                main['Securities (1Y-3Y)']      + 
                                main['Securities (3Y-5Y)']      + 
                                main['Securities (5Y-15Y)']     + 
                                main['Securities (15Y+)']
                            )) / 12

print(main['Security Maturity'].describe())

# create the 'Loan Maturity' variable:
main['Loan Maturity'] = ((
                            main['Loans (3M-)']*1.5 + 
                            main['Loans (3M-1Y)']*7.5 + 
                            main['Loans (1Y-3Y)']*24 + 
                            main['Loans (3Y-5Y)']*48 + 
                            main['Loans (5Y-15Y)']*120 + 
                            main['Loans (15Y+)']*180
                            ) / (
                                main['Loans (3M-)']        + 
                                main['Loans (3M-1Y)']      + 
                                main['Loans (1Y-3Y)']      + 
                                main['Loans (3Y-5Y)']      + 
                                main['Loans (5Y-15Y)']     + 
                                main['Loans (15Y+)']
                            )) / 12

print(main['Loan Maturity'].describe())

In [ ]:
# compute the 'Asset Maturity' variable. This is the weighted average of the maturity of securities and loans:
main['Asset Maturity'] = (main['Security Maturity']*main['Total Securities'] + main['Loan Maturity']*main['Total Loans']) / (main['Total Assets'])
print(main['Asset Maturity'].describe())

In [ ]:
main['Long Assets Maturity'] = (main['Security Maturity']*main['Total Securities'] + main['Loan Maturity']*main['Total Loans']) \
                                / (main['Total Securities'] + main['Total Loans'])
print(main['Long Assets Maturity'].describe())

###### Income Definitions

In [ ]:
# compute the variable above avoiding the SettingWithCopyWarning:
main.loc[:, 'Deposit Expenditure'] = (main['RIAD4073'].fillna(0) - main['RIAD4200'].fillna(0) - main['RIAD4185'].fillna(0) - 
                                      main['RIAD4180'].fillna(0) - main['RIAD4172'].fillna(0))

In [ ]:
main.loc[:, 'Actual Deposit Exp'] = main.groupby(['IDRSSD', 'Year'])['Deposit Expenditure'].diff().fillna(main['Deposit Expenditure'])
main.loc[:, 'Actual Deposit Exp'] = main['Actual Deposit Exp'].mask(main['Actual Deposit Exp'] <= 0, np.nan)
main['Actual Deposit Exp'].describe()

In [ ]:
# Compute deposit rates:
main.loc[:,'Deposit Rate'] = main['Actual Deposit Exp'] / main['Total Deposits']
print(main['Deposit Rate'].describe())

In [ ]:
# Create an extra column in df3 with the avg. deposit rate per Date:
main.loc[:,'Avg. Deposit Rate'] = main.groupby('Date')['Deposit Rate'].transform('mean')
main.loc[:,'Std. Deposit Rate'] = main.groupby('Date')['Deposit Rate'].transform('std')
main.loc[:,'R_hat'] = (main['Deposit Rate'] - main['Avg. Deposit Rate']) #/ main['Std. Deposit Rate']
main['R_hat'].describe()

In [ ]:
balanced_panel = 0

if balanced_panel == 1:

    # Step 1: Count the total number of unique dates in the dataset
    total_dates = main['Date'].nunique()
    print('Number of dates:', total_dates)

    # Step 2: Count the number of unique dates each bank shows up
    bank_date_counts = main.groupby('IDRSSD')['Date'].nunique()
    print('Banks that show up in all dates:', sum(bank_date_counts==total_dates))

    # Step 3: Identify banks that are present in all dates with positive 'Total Deposit':
    banks = bank_date_counts[bank_date_counts == total_dates].index
    main = main[main['IDRSSD'].isin(banks)]


## The Regional Banking Crisis (2023)

In [ ]:
df = main[['Date', 'IDRSSD', 'Financial Institution Name', 
           'Loan Maturity', 'Treasury Maturity', 'Security Maturity', 'Asset Maturity', 
           'Total Assets', 'Total Securities', 'Total Loans', 'Total Deposits', 'Cash', 
           'Securities Share', 'Loans Share', 'Cash Share'
           ]]

In [ ]:
df = df[(df['Date']=='2019-03-31') | (df['Date']=='2021-12-31')]
df = df.sort_values(by=['IDRSSD', 'Date']).reset_index(drop=True)

In [ ]:
# Get the total number of unique dates in the DataFrame
total_dates = df['Date'].nunique()

# Identify banks that appear in all dates
banks_all_dates = df.groupby('IDRSSD')['Date'].nunique() == total_dates
banks_to_keep = banks_all_dates[banks_all_dates].index

# Filter the DataFrame to keep only these banks
df = df[df['IDRSSD'].isin(banks_to_keep)].reset_index(drop=True)

In [ ]:
total_dates

In [ ]:
failed_banks = [802866, 4114567, 3437483]

In [ ]:
df['Asset Growth'] = df.groupby('IDRSSD')['Total Assets'].pct_change(fill_method=None)
df['Deposit Growth'] = df.groupby('IDRSSD')['Total Deposits'].pct_change(fill_method=None)
df['Securities Growth'] = df.groupby('IDRSSD')['Total Securities'].pct_change(fill_method=None)
df['Loans Growth'] = df.groupby('IDRSSD')['Total Loans'].pct_change(fill_method=None)
df['Delta Securities Share'] = df.groupby('IDRSSD')['Securities Share'].diff()
df['Delta Loans Share'] = df.groupby('IDRSSD')['Loans Share'].diff()
df['Delta Security Maturity'] = df.groupby('IDRSSD')['Security Maturity'].diff()

In [ ]:
df[df['IDRSSD'].isin(failed_banks)][['Date', 'Financial Institution Name',
                                     'Total Assets', 'Total Deposits', 'Total Securities', 
                                     'Asset Growth', 'Deposit Growth', 'Securities Growth', 'Loans Growth',
                                     'Delta Securities Share', 'Delta Security Maturity']]

In [ ]:
df = df[(df['Asset Growth'] > df['Asset Growth'].quantile(0.005)) & (df['Asset Growth'] < df['Asset Growth'].quantile(0.995))]

In [ ]:
df['Securities Growth'].quantile(0.97)

In [ ]:
df = df[(df['Securities Growth'] > df['Securities Growth'].quantile(0.03)) & 
        (df['Securities Growth'] < df['Securities Growth'].quantile(0.97))]

In [ ]:
plt.figure(figsize=(8, 8))
#binned_scatter(df['Deposit Growth'], df['Securities Growth'], marker='.', q=1000)
plt.scatter(df['Deposit Growth'], df['Securities Growth'], marker='.')
plt.scatter(df[df['IDRSSD'].isin(failed_banks)]['Deposit Growth'], 
            df[df['IDRSSD'].isin(failed_banks)]['Securities Growth'],
            marker='s'
            )
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Get current axis and set y-axis ticks equal to x-axis ticks
ax = plt.gca()
ticks = ax.get_yticks()
ax.set_xticks(ticks)

plt.xlabel('Deposit Growth', fontsize=14)
plt.ylabel('Securities Growth', fontsize=14)
plt.title('Deposits and Securities Growth', fontsize=16)
plt.grid(color='lightgrey', ls=':', alpha=0.5, lw=1)
# Add 45-degree line:
lims = [min(ax.get_xlim()[0], ax.get_ylim()[0]), max(ax.get_xlim()[1], ax.get_ylim()[1])]
plt.plot(lims, lims, linestyle='--', color='black', alpha=0.5)
plt.show()

plt.figure(figsize=(8, 8))
#binned_scatter(df['Deposit Growth'], df['Asset Growth'], marker='.', q=1000)
plt.scatter(df['Deposit Growth'], df['Asset Growth'], marker='.')
plt.scatter(df[df['IDRSSD'].isin(failed_banks)]['Deposit Growth'], 
            df[df['IDRSSD'].isin(failed_banks)]['Asset Growth'],
            marker='s'
            )
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
# Set y-axis ticks equal to x-axis ticks:
plt.gca().set_yticks(ticks)
plt.gca().set_xticks(ticks)
plt.xlabel('Deposit Growth', fontsize=14)
plt.ylabel('Asset Growth', fontsize=14)
plt.title('Deposits and Asset Growth', fontsize=16)
plt.grid(color='lightgrey', ls=':', alpha=0.5, lw=1)
# Add 45-degree line:
ax = plt.gca()
lims = [min(ax.get_xlim()[0], ax.get_ylim()[0]), max(ax.get_xlim()[1], ax.get_ylim()[1])]
plt.plot(lims, lims, linestyle='--', color='black', alpha=0.5)
plt.show()

plt.figure(figsize=(8, 8))
#binned_scatter(df['Deposit Growth'], df['Loans Growth'], marker='.', q=1000)
plt.scatter(df['Deposit Growth'], df['Loans Growth'], marker='.')
plt.scatter(df[df['IDRSSD'].isin(failed_banks)]['Deposit Growth'], 
            df[df['IDRSSD'].isin(failed_banks)]['Loans Growth'],
            marker='s'
            )
# Set y-axis ticks equal to x-axis ticks:
plt.gca().set_yticks(ticks)
plt.gca().set_xticks(ticks)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Deposit Growth', fontsize=14)
plt.ylabel('Loans Growth', fontsize=14)
plt.title('Deposits and Loans Growth', fontsize=16)
plt.grid(color='lightgrey', ls=':', alpha=0.5, lw=1)

plt.xlim(-1.2,6.5)
plt.ylim(-1.2,6.5)

# Set both x and y ticks to be the same
plt.xticks(ticks, fontsize=14)
plt.yticks(ticks, fontsize=14)
plt.gca().set_xticks(ticks)
plt.gca().set_yticks(ticks)

# Add 45-degree line
ax = plt.gca()
lims = [min(ax.get_xlim()[0], ax.get_ylim()[0]), max(ax.get_xlim()[1], ax.get_ylim()[1])]
plt.plot(lims, lims, linestyle='--', color='black', alpha=0.5)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
binned_scatter(df['Deposit Growth'], df['Delta Loans Share'], marker='.', q=1000)
plt.scatter(df[df['IDRSSD'].isin(failed_banks)]['Deposit Growth'], 
            df[df['IDRSSD'].isin(failed_banks)]['Delta Loans Share'],
            marker='s'
            )
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Deposit Growth', fontsize=14)
plt.ylabel('$\Delta$Loans Share', fontsize=14)
plt.title('Deposits and $\Delta$Loans Share', fontsize=16)
plt.grid(color='lightgrey', ls=':', alpha=0.5, lw=1)
plt.show()

plt.figure(figsize=(12, 6))
binned_scatter(df['Deposit Growth'], df['Delta Securities Share'], marker='.', q=1000)
plt.scatter(df[df['IDRSSD'].isin(failed_banks)]['Deposit Growth'], 
            df[df['IDRSSD'].isin(failed_banks)]['Delta Securities Share'],
            marker='s'
            )
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Deposit Growth', fontsize=14)
plt.ylabel('$\Delta$Securities Share', fontsize=14)
plt.title('Deposits and $\Delta$Securities Share', fontsize=16)
plt.grid(color='lightgrey', ls=':', alpha=0.5, lw=1)
plt.show()

plt.figure(figsize=(12, 6))
#binned_scatter(df['Deposit Growth'], df['Delta Security Maturity'], marker='.', q=1000)
plt.scatter(df['Deposit Growth'], df['Delta Security Maturity'], marker='.')
plt.scatter(df[df['IDRSSD'].isin(failed_banks)]['Deposit Growth'], 
            df[df['IDRSSD'].isin(failed_banks)]['Delta Security Maturity'],
            marker='s'
            )
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Deposit Growth', fontsize=14)
plt.ylabel('$\Delta$Security Maturity', fontsize=14)
plt.title('Deposits and $\Delta$ Security Maturity', fontsize=16)
plt.grid(color='lightgrey', ls=':', alpha=0.5, lw=1)
plt.show()

### SVB

In [ ]:
# ------------------------------- START DATE ------------------------------
svb_start_date = main[main['IDRSSD']==802866]['Date'].min()
svb_start = main[main['Date'] == svb_start_date]
svb_start_share = svb_start[svb_start['IDRSSD'] == 802866]['Securities Share'].values[0]
svb_start.loc[:, 'Log Assets'] = np.log(svb_start['Total Assets'])
# ------------------------------- END DATE --------------------------------
svb_date = main[(main['Date'] == '2022-12-31')]
# create a log of the total assets using .loc:
svb_date.loc[:, 'Log Assets'] = np.log(svb_date['Total Assets'])
svb_share = svb_date[svb_date['IDRSSD'] == 802866]['Securities Share'].values[0]
svb_share_cash = svb_date[svb_date['IDRSSD'] == 802866]['Cash Share'].values[0]
svb_share_loans = svb_date[svb_date['IDRSSD'] == 802866]['Loans Share'].values[0]
# ------------------------------ PLOTS -------------------------------------
sns.histplot(data=svb_start, x='Securities Share', bins=25, kde=True, stat='density', label='2001-03-31')
# plot a vertical line at the ID=802866 (SVB):
plt.axvline(x=svb_start_share, linestyle='--', linewidth=2, color='blue')
sns.histplot(data=svb_date, x='Securities Share', bins=25, kde=True, stat='density', label='2022-12-31')
# plot a vertical line at the ID=802866 (SVB):
plt.axvline(x=svb_date[svb_date['IDRSSD'] == 802866]['Securities Share'].values[0], linestyle='--', linewidth=2, color='orange')
plt.title(f'Distribution of Securities Share')
plt.legend()
plt.show()

sns.histplot(data=svb_start, x='Log Assets', bins=25, kde=True, stat='density', label='2001-03-31')
# plot a vertical line at the ID=802866 (SVB):
plt.axvline(x=svb_start[svb_start['IDRSSD'] == 802866]['Log Assets'].values[0], color='blue', linestyle='--', linewidth=2)
sns.histplot(data=svb_date, x='Log Assets', bins=25, kde=True, stat='density', label='2022-12-31')
# plot a vertical line at the ID=802866 (SVB):
plt.axvline(x=svb_date[svb_date['IDRSSD'] == 802866]['Log Assets'].values[0], color='orange', linestyle='--', linewidth=2)
plt.title('Distribution of Log Total Assets')
plt.legend()
plt.show()

# plot the distribution of the securities share for the banks with log assets larger than SVB:
sns.histplot(data=svb_start[svb_start['Log Assets'] >= svb_start[svb_start['IDRSSD'] == 802866]['Log Assets'].values[0]],
              x='Securities Share', bins=5, kde=False, stat='density', label='2001-03-31')
plt.axvline(x=svb_start[svb_start['IDRSSD'] == 802866]['Securities Share'].values[0], color='blue', linestyle='--', linewidth=2)
sns.histplot(data=svb_date[svb_date['Log Assets'] >= svb_date[svb_date['IDRSSD'] == 802866]['Log Assets'].values[0]],
              x='Securities Share', bins=5, kde=False, stat='density', label='2022-12-31')
plt.axvline(x=svb_date[svb_date['IDRSSD'] == 802866]['Securities Share'].values[0], color='orange', linestyle='--', linewidth=2)
plt.title('Distribution of Securities Share for Banks with Log Assets >= SVB')
plt.xlabel('Securities Share')
plt.ylabel('Density')
plt.legend()
plt.show()

In [ ]:
# print the banks that had a larger securities share than SVB and longer asset maturity:
svb_date[#(svb_date['Securities Share'] >= svb_share) & 
         #(svb_date['Asset Maturity'] >= svb_date[svb_date['IDRSSD'] == 802866]['Asset Maturity'].values[0]) & 
         (svb_date['Log Assets'] >= svb_date[svb_date['IDRSSD'] == 802866]['Log Assets'].values[0])][['Financial Institution Name', 'Log Assets', 
                                                                                                      'Securities Share', 'Cash Share',
                                                                                                      'Security Maturity', 'Asset Maturity']].sort_values('Securities Share', ascending=False)

In [ ]:
# print the banks that had a larger securities share than SVB and longer asset maturity:
svb_date[(svb_date['Securities Share'] >= svb_share) & 
         #(svb_date['Asset Maturity'] >= svb_date[svb_date['IDRSSD'] == 802866]['Asset Maturity'].values[0]) & 
         (svb_date['Log Assets'] >= svb_date[svb_date['IDRSSD'] == 802866]['Log Assets'].values[0])][['Financial Institution Name', 'Log Assets', 
                                                                                                      'Securities Share', 'Cash Share',
                                                                                                      'Security Maturity', 'Asset Maturity']].sort_values('Log Assets', ascending=False)

## Aggregate Data

In [ ]:
# Load monetary policy shock data:
mp_shocks = pd.read_csv('brw-shock-series.csv')

# Drop column that has 'Unnamed' in it:
mp_shocks = mp_shocks.loc[:, ~mp_shocks.columns.str.contains('^Unnamed')]

# keep only the first 4 columns:
mp_shocks = mp_shocks.iloc[:, 0:4]

# rename columns:
mp_shocks.columns = mp_shocks.columns.str.replace(' (updated)', '')
mp_shocks.loc[:, 'month'] = mp_shocks['month'].str.replace('m', '-')
mp_shocks = mp_shocks[['month', 'BRW_monthly']]

# drop the 'NaT' row:
mp_shocks = mp_shocks.dropna()
mp_shocks['month'] = pd.to_datetime(mp_shocks['month'], format='%Y-%m')
# compute the quarterly average of the monthly shocks:
mp_shocks['quarter'] = mp_shocks['month'].dt.to_period('Q')
mp_shocks['BRW_quarterly'] = mp_shocks.groupby('quarter')['BRW_monthly'].transform('sum')

# keep only one observation for 'month' and 'BRW_quarterly' for each quarter:
mp_shocks = mp_shocks.drop_duplicates(subset='quarter', keep='first')
mp_shocks['Date'] = mp_shocks['month'] - pd.DateOffset(days = 1)

mp_shocks = mp_shocks[['Date', 'BRW_quarterly']]
mp_shocks.rename(columns={'BRW_quarterly': 'MP Shocks'}, inplace=True)

mp_shocks.loc[:,'Positive FF Shock'] = 0
mp_shocks.loc[mp_shocks['MP Shocks'] > 0, 'Positive FF Shock'] = 1

In [ ]:
# Load the Excess Bond Premium data:
ebp = pd.read_csv('ebp_csv.csv')

In [ ]:
# create a quarter column:
ebp['quarter'] = pd.to_datetime(ebp['date']).dt.to_period('Q')

In [ ]:
# generate ebp quarterly averages:
ebp['ebp_q'] = ebp.groupby('quarter')['ebp'].transform('mean')

In [ ]:
# load fredgraph data:
aggregates = pd.read_csv('fredgraph.csv')

# rename some variables
aggregates.rename(columns = {'DATE':'Date', 
                             'NGDPSAXDCUSQ': 'Nominal GDP', 
                             'CPIAUCSL_NBD19840101': 'CPI',
                             'GDPDEF': 'Deflator'}, inplace = True)

# drop the first row:
aggregates = aggregates.iloc[1:,:]

# for all entries with '.' substitute with NaN:
aggregates = aggregates.replace('.', np.nan)

# make 'Deflator' and 'Nominal GDP' floats:
aggregates['Deflator'] = aggregates['Deflator'].astype(float)
aggregates['Nominal GDP'] = aggregates['Nominal GDP'].astype(float)
aggregates['FEDFUNDS'] = aggregates['FEDFUNDS'].astype(float)
aggregates['CPI'] = aggregates['CPI'].astype(float)

# make 'Date' a datetime object:
aggregates['Date'] = pd.to_datetime(aggregates['Date'])

# create 'Real GDP' variable:
aggregates['Real GDP'] = aggregates['Nominal GDP'] / aggregates['CPI'] * 100
aggregates = aggregates.dropna()

# adjust the FF rate: 
aggregates['FEDFUNDS'] = aggregates['FEDFUNDS'] / 100
aggregates['Date'] = aggregates['Date'] - pd.DateOffset(days = 1)

# create 'Inflation' as the growth rate of the CPI:
aggregates['Inflation'] = aggregates['CPI'].pct_change(fill_method=None)

# create a variable named 'High Inflation' as 1 if the inflation is above the average, and 0 otherwise:
aggregates['High Inflation'] = 0
aggregates.loc[aggregates['Inflation'] > aggregates['Inflation'].mean(), 'High Inflation'] = 1

# create a variable named 'FF Hike' as 1 if the FF rate increased, and 0 otherwise:
aggregates['FF Hike'] = 0
aggregates.loc[aggregates['FEDFUNDS'] > aggregates['FEDFUNDS'].shift(1), 'FF Hike'] = 1

param = 1600

# De-trend the variables using an HP filter:
aggregates['De-trended Real GDP'] = sm.tsa.filters.hpfilter(np.log(aggregates['Real GDP']), lamb=param)[0]

# create a variable named 'Boom' that is 1, if the De-trended Real GDP is above 0, and 0 otherwise:
aggregates['Recession'] = 0
aggregates.loc[aggregates['De-trended Real GDP'] < 0, 'Recession'] = 1

In [ ]:
# load data on mergers and acquisitions:
transformations = pd.read_csv('CSV_TRANSFORMATIONS.csv')

# rename the columns:
transformations = transformations.rename(columns={'#ID_RSSD_PREDECESSOR': 'Predecessor ID', 
                                                  'ID_RSSD_SUCCESSOR': 'Successor ID',
                                                 'DT_TRANS': 'Transaction Date',
                                                 'TRNSFM_CD': 'Transaction Code',})

# convert the 'transaction_date' to datetime format:
transformations['Transaction Date'] = pd.to_datetime(transformations['Transaction Date'], format='%Y%m%d')
transformations = transformations[['Predecessor ID', 'Successor ID', 'Transaction Date', 'Transaction Code']]

In [ ]:
#main = pd.merge(main, aggregates, how = 'left', on = 'Date').sort_values(by = ['IDRSSD', 'Date'])
#main = pd.merge(main, mp_shocks, how = 'left', on = 'Date').sort_values(by = ['IDRSSD', 'Date'])

## Start Analysis

### Time-series analysis (portfolio shares)

In [ ]:
df = main[['Date', 'IDRSSD', 'Financial Institution Name', 
           'Loan Maturity', 'Treasury Maturity', 'Security Maturity', 'Asset Maturity', 'Long Assets Maturity',
           'Total Assets', 'Total Securities', 'Total Loans', 'Total Deposits', 'Cash', 
           'Securities Share', 'Loans Share', 'Cash Share',
           'Deposit Rate', 'Avg. Deposit Rate', 'Std. Deposit Rate', 'R_hat',
           ]]

# Assuming df is your DataFrame
df = df.dropna(subset=['Securities Share', 'Loans Share', 'Cash Share', 'Total Assets',
                       'Asset Maturity', 'Loan Maturity', 'Security Maturity', 'Total Loans', 'Total Securities'])

In [ ]:
df = df[df['Date'].dt.year<2020]

In [ ]:
# drop the bottom 20% of the banks in terms of total assets per 'Date':
df = df.groupby('Date').apply(lambda x: x[x['Total Assets'] > x['Total Assets'].quantile(0.20)]).reset_index(drop=True)

# use nlargest to drop the 10 largest banks in 'Total Assets' per 'Date':
df = df.groupby('Date').apply(lambda x: x[~x['IDRSSD'].isin(x.nlargest(10, 'Total Assets')['IDRSSD'])]).reset_index(drop=True)

In [ ]:

# Compute weighted averages
df['Weighted_Securities_Share'] = df['Securities Share'] * df['Total Assets']
df['Weighted_Loans_Share'] = df['Loans Share'] * df['Total Assets']
df['Weighted_Cash_Share'] = df['Cash Share'] * df['Total Assets']
df['Weighted_Asset_Maturity'] = df['Asset Maturity'] * df['Total Assets']
df['Weighted_Loans_Maturity'] = df['Loan Maturity'] * df['Total Assets']
df['Weighted_Securities_Maturity'] = df['Security Maturity'] * df['Total Assets']

# Group by 'Date' and compute the sum of weighted shares and total assets
grouped = df.groupby('Date').agg({
    'Weighted_Securities_Share': 'sum',
    'Weighted_Loans_Share': 'sum',
    'Weighted_Cash_Share': 'sum',
    'Weighted_Asset_Maturity': 'sum',
    'Weighted_Loans_Maturity': 'sum',
    'Weighted_Securities_Maturity': 'sum',
    'Total Assets': 'sum',
    'Total Loans': 'sum',
    'Total Securities': 'sum'
}).reset_index()

# Compute the weighted averages
grouped['Securities_Share_Avg'] = grouped['Weighted_Securities_Share'] / grouped['Total Assets']
grouped['Loans_Share_Avg'] = grouped['Weighted_Loans_Share'] / grouped['Total Assets']
grouped['Cash_Share_Avg'] = grouped['Weighted_Cash_Share'] / grouped['Total Assets']
grouped['Asset_Maturity_Avg'] = grouped['Weighted_Asset_Maturity'] / grouped['Total Assets']
grouped['Loans_Maturity_Avg'] = grouped['Weighted_Loans_Maturity'] / grouped['Total Assets']
grouped['Securities_Maturity_Avg'] = grouped['Weighted_Securities_Maturity'] / grouped['Total Assets']

# Plot the time series
plt.figure(figsize=(12, 6))
plt.plot(grouped['Date'], grouped['Securities_Share_Avg'], label='Securities Share', ls='-')
plt.plot(grouped['Date'], grouped['Loans_Share_Avg'], label='Loans Share', ls='--')
plt.plot(grouped['Date'], grouped['Cash_Share_Avg'], label='Cash Share', ls=':')

plt.title('Time Series of Weighted Securities, Loans, and Cash Shares')
plt.xlabel('Date')
plt.ylabel('Weighted Share')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5, linewidth=0.5, color='lightgrey')
plt.show()

# Plot the time series
plt.figure(figsize=(12, 6))
plt.plot(grouped['Date'], grouped['Securities_Maturity_Avg'], label='Securities', ls='-', lw=3)
#plt.plot(grouped['Date'], grouped['Loans_Maturity_Avg'], label='Loans', ls='--')
# increase the size of the ticks and labels:
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
#plt.plot(grouped['Date'], grouped['Asset_Maturity_Avg'], label='Asset', ls='-', lw=3)
plt.title('Time Series of Weighted Securities Maturity', fontsize=18)
plt.xlabel('Date', fontsize=16)
plt.ylabel('Weighted Securities Maturity', fontsize=16)
plt.grid(True, linestyle='--', alpha=0.5, linewidth=0.5, color='lightgrey')
plt.show()

# Plot the time series
plt.figure(figsize=(12, 6))
plt.plot(grouped['Date'], grouped['Asset_Maturity_Avg'], label='Securities', ls='-', lw=3)
#plt.plot(grouped['Date'], grouped['Loans_Maturity_Avg'], label='Loans', ls='--')
# increase the size of the ticks and labels:
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
#plt.plot(grouped['Date'], grouped['Asset_Maturity_Avg'], label='Asset', ls='-', lw=3)
plt.title('Time Series of Weighted Asset Maturity', fontsize=18)
plt.xlabel('Date', fontsize=16)
plt.ylabel('Weighted Asset Maturity', fontsize=16)
plt.grid(True, linestyle='--', alpha=0.5, linewidth=0.5, color='lightgrey')
plt.show()

In [ ]:
print(grouped[grouped['Date'] == grouped['Date'].min()][['Date', 'Loans_Share_Avg', 
                                                                        'Securities_Share_Avg', 'Cash_Share_Avg']])
print(grouped[grouped['Date'] == grouped['Date'].max()][['Date', 'Loans_Share_Avg', 
                                                                        'Securities_Share_Avg', 'Cash_Share_Avg']])

In [ ]:
print(grouped[grouped['Date'] == grouped['Date'].min()][['Date', 'Asset_Maturity_Avg', 
                                                                        'Securities_Maturity_Avg', 'Loans_Maturity_Avg']])
print(grouped[grouped['Date'] == grouped['Date'].max()][['Date', 'Asset_Maturity_Avg', 
                                                                        'Securities_Maturity_Avg', 'Loans_Maturity_Avg']])

In [ ]:
df = main[['Date', 'IDRSSD', 'Financial Institution Name', 
           'Loan Maturity', 'Treasury Maturity', 'Security Maturity', 'Asset Maturity', 'Long Assets Maturity',
           'Total Assets', 'Total Securities', 'Total Loans', 'Total Deposits', 'Cash', 
           'Securities Share', 'Loans Share', 'Cash Share',
           'Deposit Rate', 'Avg. Deposit Rate', 'Std. Deposit Rate', 'R_hat',
           ]]

In [ ]:
# Assuming df is your DataFrame
df = df.dropna(subset=['Securities Share', 'Loans Share', 'Cash Share', 'Total Assets',
                       'Asset Maturity', 'Loan Maturity', 'Security Maturity', 'Total Loans', 'Total Securities'])

# Group by 'Date' and compute the sum of weighted shares and total assets
grouped = df.groupby('Date').agg({
    'Securities Share': 'mean',
    'Loans Share': 'mean',
    'Cash Share': 'mean',
    'Asset Maturity': 'mean',
    'Loan Maturity': 'mean',
    'Security Maturity': 'mean',
    'Total Assets': 'mean',
    'Total Loans': 'mean',
    'Total Securities': 'mean'
}).reset_index()

# Plot the time series
plt.figure(figsize=(12, 6))
plt.plot(grouped['Date'], grouped['Securities Share'], label='Securities Share', ls='-')
plt.plot(grouped['Date'], grouped['Loans Share'], label='Loans Share', ls='--')
plt.plot(grouped['Date'], grouped['Cash Share'], label='Cash Share', ls=':')

plt.title('Time Series of Avg. Securities, Loans, and Cash Shares')
plt.xlabel('Date')
plt.ylabel('Avg. Share')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5, linewidth=0.5, color='lightgrey')
plt.show()

# Plot the time series
plt.figure(figsize=(12, 6))
plt.plot(grouped['Date'], grouped['Security Maturity'], label='Securities', ls='-', lw=3)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.title('Time Series of Avg. Securities Maturity', fontsize=18)
plt.xlabel('Date', fontsize=16)
plt.ylabel('Avg. Securities Maturity', fontsize=16)
plt.grid(True, linestyle='--', alpha=0.5, linewidth=0.5, color='lightgrey')
plt.show()

# Plot the time series
plt.figure(figsize=(12, 6))
plt.plot(grouped['Date'], grouped['Asset Maturity'], label='Asset', ls='-', lw=3)
# increase the size of the ticks and labels:
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.title('Time Series of Avg. Asset Maturity', fontsize=18)
plt.xlabel('Date', fontsize=16)
plt.ylabel('Avg. Asset Maturity', fontsize=16)
plt.grid(True, linestyle='--', alpha=0.5, linewidth=0.5, color='lightgrey')
plt.show()

In [ ]:
print(grouped[grouped['Date'] == grouped['Date'].min()][['Date', 'Loans Share', 
                                                                        'Securities Share', 'Cash Share']])
print(grouped[grouped['Date'] == grouped['Date'].max()][['Date', 'Loans Share', 
                                                                        'Securities Share', 'Cash Share']])


In [ ]:
print(grouped[grouped['Date'] == grouped['Date'].min()][['Date', 'Asset Maturity', 
                                                                        'Security Maturity', 'Loan Maturity']])
print(grouped[grouped['Date'] == grouped['Date'].max()][['Date', 'Asset Maturity', 
                                                                        'Security Maturity', 'Loan Maturity']])

In [ ]:
# create 'base_date' variable:
base_date = '2021-12-31'

# what is the ID of the bank with more assets in 2001:
top = 5
top_id_2001 = df[df['Date'] == base_date].sort_values(by = 'Total Assets', ascending = False).head(top)['Financial Institution Name']

In [ ]:
# plot the time series of portfolio shares for top_10_id_2001 using seaborn:
plt.figure(figsize=(12, 6))
sns.scatterplot(data=df[df['Financial Institution Name'].isin(top_id_2001)], x='Date',
                 y='Securities Share', hue='Financial Institution Name', palette='tab10', 
             marker='o')
plt.title(f'Time Series of Securities Share for Top {top} Banks by Assets in {base_date}', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Securities Share', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.5, linewidth=0.5, color='lightgrey')
# let the legend be outside the plot:
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

# plot the time series of portfolio shares for top_10_id_2001 using seaborn:
plt.figure(figsize=(12, 6))
sns.scatterplot(data=df[df['Financial Institution Name'].isin(top_id_2001)], x='Date', y='Loans Share', 
                hue='Financial Institution Name', palette='tab10', 
             marker='o')
plt.title(f'Time Series of Loans Share for Top {top} Banks by Assets in {base_date}', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Loans Share', fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(True, linestyle='--', alpha=0.5, linewidth=0.5, color='lightgrey')
# let the legend be outside the plot:
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14)
plt.show()

# plot the time series of portfolio shares for top_10_id_2001 using seaborn:
plt.figure(figsize=(12, 6))
sns.scatterplot(data=df[df['Financial Institution Name'].isin(top_id_2001)], x='Date', y='Cash Share',
                 hue='Financial Institution Name', palette='tab10', 
             marker='o')
plt.title(f'Time Series of Cash Share for Top {top} Banks by Assets in {base_date}', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Cash Share', fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(True, linestyle='--', alpha=0.5, linewidth=0.5, color='lightgrey')
# let the legend be outside the plot:
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14)
plt.show()



In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(data=df[df['Financial Institution Name'].isin(top_id_2001)], x='Date', y='Asset Maturity', hue='Financial Institution Name',
                 palette='tab10', 
             marker='o')
plt.title(f'Time Series of Asset Maturity for Top {top} Banks by Assets in {base_date}', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Asset Maturity', fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(True, linestyle='--', alpha=0.5, linewidth=0.5, color='lightgrey')
# let the legend be outside the plot:
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14)
plt.show()

plt.figure(figsize=(12, 6))
sns.scatterplot(data=df[df['Financial Institution Name'].isin(top_id_2001)], x='Date', y='Loan Maturity', hue='Financial Institution Name',
                 palette='tab10', 
             marker='o')
plt.title(f'Time Series of Loan Maturity for Top {top} Banks by Assets in {base_date}', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Loans Maturity', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.5, linewidth=0.5, color='lightgrey')
# let the legend be outside the plot:
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14)
plt.show()

plt.figure(figsize=(12, 6))
sns.scatterplot(data=df[df['Financial Institution Name'].isin(top_id_2001)], x='Date', y='Security Maturity', hue='Financial Institution Name',
                 palette='tab10', 
             marker='o')
plt.title(f'Time Series of Securities Maturity for Top {top} Banks by Assets in {base_date}', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Securities Maturity', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.5, linewidth=0.5, color='lightgrey')
# let the legend be outside the plot:
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=14)
plt.show()


### Cross-sectional analysis (portfolio shares)

In [ ]:
df = main[['Date', 'IDRSSD', 'Financial Institution Name', 
           'Loan Maturity', 'Treasury Maturity', 'Security Maturity', 'Asset Maturity', 'Long Assets Maturity',
           'Treasury Maturity', 'MBS Maturity',
           'Total Assets', 'Total Securities', 'Total Loans', 'Total Deposits', 'Cash', 
           'Securities Share', 'Loans Share', 'Cash Share',
           'Deposit Rate', 'Avg. Deposit Rate', 'Std. Deposit Rate', 'R_hat',
           ]]

In [ ]:
print(df['Cash Share'].quantile([0.01, 0.99]))
print(df['Securities Share'].quantile([0.01, 0.99]))
print(df['Loans Share'].quantile([0.01, 0.99]))
print(df['Asset Maturity'].quantile([0.01, 0.99]))
print(df['Security Maturity'].quantile([0.01, 0.99]))
print(df['Loan Maturity'].quantile([0.01, 0.99]))

# drop the top 1% and bottom 1% of the data for the variables above:
df = df[(df['Cash Share'] > df['Cash Share'].quantile(0.01)) & (df['Cash Share'] < df['Cash Share'].quantile(0.99))]
df = df[(df['Securities Share'] > df['Securities Share'].quantile(0.01)) & (df['Securities Share'] < df['Securities Share'].quantile(0.99))]
df = df[(df['Loans Share'] > df['Loans Share'].quantile(0.01)) & (df['Loans Share'] < df['Loans Share'].quantile(0.99))]
df = df[(df['Asset Maturity'] > df['Asset Maturity'].quantile(0.01)) & (df['Asset Maturity'] < df['Asset Maturity'].quantile(0.99))]
df = df[(df['Security Maturity'] > df['Security Maturity'].quantile(0.01)) & (df['Security Maturity'] < df['Security Maturity'].quantile(0.99))]
df = df[(df['Loan Maturity'] > df['Loan Maturity'].quantile(0.01)) & (df['Loan Maturity'] < df['Loan Maturity'].quantile(0.99))]
df = df.dropna(subset=['Securities Share', 'Loans Share', 'Cash Share', 'Total Assets',
                       'Asset Maturity', 'Loan Maturity', 'Security Maturity', 'Total Loans', 'Total Securities'])

In [ ]:
# create the 'Log Total Assets' variable:
df['Log Total Assets'] = np.log(df['Total Assets'])

In [ ]:
# use seaborn to plot the distribution of 'Security Maturity':
plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='Loan Maturity', bins=50, kde=False, stat='density', label='Loans')
sns.histplot(data=df, x='Security Maturity', bins=50, kde=False, stat='density', label='Securities', 
             alpha=0.5)
plt.legend(fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel('Density', fontsize=16)
plt.xlabel('Maturity (years)', fontsize=16)
# write the averages for each variable in the plot:
plt.text(x=11, y=0.14, s=f'Loans: {np.round(df["Loan Maturity"].mean(),2)}', color='blue', alpha=0.8, fontsize=16)
plt.text(x=11, y=0.13, s=f'Securities: {np.round(df["Security Maturity"].mean(),2)}', color='orange', fontsize=16)
plt.show()

print(df['Security Maturity'].describe())
print(df['Loan Maturity'].describe())

In [ ]:
plt.figure(figsize=(12, 6))
binned_scatter(
df['Log Total Assets'], 
df['Cash Share'], 
q=100, 
#color='blue', 
)

# Plot settings
plt.title('Binned Log Total Assets vs. Cash Share (Quantile Bins)') 
plt.ylabel('Cash Share')
plt.xlabel('Log Total Assets')
#plt.legend()
plt.grid(True, which='both', linestyle='--', lw=0.5, alpha=0.5, color='lightgrey')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
binned_scatter(
df['Log Total Assets'], 
df['Loans Share'], 
q=100, 
#color='blue', 
)


# Plot settings
plt.title('Binned Log Total Assets vs. Loans Share (Quantile Bins)') 
plt.ylabel('Loans Share')
plt.xlabel('Log Total Assets')
#plt.legend()
plt.grid(True, which='both', linestyle='--', lw=0.5, alpha=0.5, color='lightgrey')
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
binned_scatter(
df['Log Total Assets'], 
df['Securities Share'], 
q=100, 
#color='blue', 
)


# Plot settings
plt.title('Binned Log Total Assets vs. Securities Share (Quantile Bins)') 
plt.ylabel('Securities Share')
plt.xlabel('Log Total Assets')
#plt.legend()
plt.grid(True, which='both', linestyle='--', lw=0.5, alpha=0.5, color='lightgrey')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
binned_scatter(
df['Log Total Assets'], 
df['Asset Maturity'], 
q=100, 
#color='blue', 
)


# Plot settings
plt.title('Binned Log Total Assets vs. Asset Maturity (Quantile Bins)') 
plt.ylabel('Asset Maturity (years)')
plt.xlabel('Log Total Assets')
plt.legend()
plt.grid(True, which='both', linestyle='--', lw=0.5, alpha=0.5, color='lightgrey')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
binned_scatter(
df['Log Total Assets'], 
df['Security Maturity'], 
q=100, 
#color='blue', 
)

# Plot settings
plt.title('Binned Log Total Assets vs. Security Maturity (Quantile Bins)') 
plt.ylabel('Security Maturity (years)')
plt.xlabel('Log Total Assets')
plt.grid(True, which='both', linestyle='--', lw=0.5, alpha=0.5, color='lightgrey')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
binned_scatter(
df['Log Total Assets'], 
df['Loan Maturity'], 
q=100, 
#color='blue', 
)


# Plot settings
plt.title('Binned Log Total Assets vs. Loans Maturity (Quantile Bins)') 
plt.ylabel('Loan Maturity (years)')
plt.xlabel('Log Total Assets')
plt.grid(True, which='both', linestyle='--', lw=0.5, alpha=0.5, color='lightgrey')
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
binned_scatter(
df['Security Maturity'], 
df['Loan Maturity'], 
q=100, 
#color='blue', 
)


# Plot settings
plt.title('Binned Security Maturity vs. Loans Maturity (Quantile Bins)', fontsize=16) 
plt.ylabel('Loan Maturity (years)', fontsize=16)
plt.xlabel('Security Maturity (years)', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.grid(True, which='both', linestyle='--', lw=0.5, alpha=0.5, color='lightgrey')
plt.legend(fontsize=14)
plt.show()

In [ ]:
# regress the 'Asset Maturity' on the 'Log Total Assets' and the squared of 'Log Total Assets'. Deal with NaNs:
df2 = df.dropna(subset=['Asset Maturity', 'Log Total Assets'])
X = df2['Log Total Assets']
X = sm.add_constant(X)

y = df2['Asset Maturity']

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

In [ ]:
# regress the 'Asset Maturity' on the 'Log Total Assets' and the squared of 'Log Total Assets'. Deal with NaNs:
df2 = df.dropna(subset=['Cash Share', 'Log Total Assets'])
X = df2['Log Total Assets']
X = sm.add_constant(X)

y = df2['Cash Share']

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

In [ ]:
# regress the 'Asset Maturity' on the 'Log Total Assets' and the squared of 'Log Total Assets'. Deal with NaNs:
df2 = df.dropna(subset=['Loans Share', 'Log Total Assets'])
X = df2['Log Total Assets']
X = sm.add_constant(X)

y = df2['Loans Share']

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

In [ ]:
# regress the 'Asset Maturity' on the 'Log Total Assets' and the squared of 'Log Total Assets'. Deal with NaNs:
df2 = df.dropna(subset=['Securities Share', 'Log Total Assets'])
X = df2['Log Total Assets']
X = sm.add_constant(X)

y = df2['Securities Share']

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

### Previous cross-sectional analysis (growth rates)

In [ ]:
def compute_growth(df, transformations, window=1):
    """
    Compute the log difference of deposits while accounting for mergers, acquisitions, and failures.

    Args:
    - df (pd.DataFrame): Dataframe containing bank-level balance sheet data.
    - transformations (pd.DataFrame): Dataframe containing information about transformations.
    - window (int): Number of quarters before and after a transaction to set as NaN for growth rate.

    Returns:
    - pd.DataFrame: The updated 'df' with a 'deposit_growth' column.
    """
    # Sort the dataframe by bank ID and date to ensure proper lag calculation
    df = df.sort_values(by=['IDRSSD', 'Date'])
    df['Real Deposits'] = df['Total Deposits'] / df['CPI'] * 100
    df['Real Loans'] = df['Total Loans'] / df['CPI'] * 100


    # Calculate log difference (log growth rate) of deposits
    df['Deposit_Growth'] = df.groupby('IDRSSD')['Real Deposits'].transform(lambda x: np.log(x).diff())
    df['Loan_Growth'] = df.groupby('IDRSSD')['Real Loans'].transform(lambda x: np.log(x).diff())
    
    # Create a mask to identify rows to be set to NaN
    for _, row in transformations.iterrows():
        predecessor_id = row['Predecessor ID']
        successor_id = row['Successor ID']
        transaction_date = row['Transaction Date']
        
        # Identify date range around the transaction
        start_date = transaction_date - pd.DateOffset(months=3 * window)
        end_date = transaction_date + pd.DateOffset(months=3 * window)
        
        # Apply NaN to the specified window for predecessor and successor banks
        df.loc[
            ((df['IDRSSD'] == predecessor_id) | (df['IDRSSD'] == successor_id)) & 
            (df['Date'].between(start_date, end_date)),
            'Deposit_Growth'
        ] = np.nan

        df.loc[
            ((df['IDRSSD'] == predecessor_id) | (df['IDRSSD'] == successor_id)) & 
            (df['Date'].between(start_date, end_date)),
            'Loan_Growth'
        ] = np.nan
    
    return df

In [ ]:
df = main[[ 'IDRSSD', 'Date', 'Financial Institution Name',             # Dates and IDS 
            'Total Deposits', 'Deposit Share', 'Total Assets',          # Bank specific quantity variables
            'Total Loans',
            'Deposit Rate', 'Deposit Spread',                           # Bank specific rate variable
            'Cash', 'Securities AC',                                    # Bank specific asset variables
            'FEDFUNDS', 'High Inflation', 'FF Hike', 'Recession',
            'CPI', 'Positive FF Shock'                                  # Macro variables
            ]]

In [ ]:
df.loc[:,'Securities Share'] = df['Securities AC'] / df['Total Assets']

In [ ]:
svb_securities = df[(df['IDRSSD'] == 802866) & (df['Date'] == '2022-12-31')]['Securities Share'].values[0]

In [ ]:
# tell me the IDs of the banks that have a 'Securities Share' above svb_securities:
right_tail_banks = df[(df['Date'] == '2022-12-31') & 
                      (df[df['Date'] == '2022-12-31']['Securities Share'] > svb_securities)]['IDRSSD'].unique()

In [ ]:
right_tail_df = df[ (df['Date'] == '2022-12-31') & 
    (df['IDRSSD'].isin(right_tail_banks))][['Financial Institution Name', 'Total Assets', 'Total Deposits']]

In [ ]:
df[(df['Date'] == '2022-12-31')][['Financial Institution Name', 'Total Assets', 'Total Deposits']].describe()

In [ ]:
right_tail_df.describe()

In [ ]:
# plot the distribution of securities share for 03/31/2023:
sns.histplot(df[df['Date'] == '2022-12-31']['Securities Share'], bins=20, kde=True)
# add a vertical line for the bank with IDRSSD=802866:
plt.axvline(x=df[(df['IDRSSD'] == 802866) & (df['Date'] == '2022-12-31')]['Securities Share'].values[0], 
            color='red', linestyle='--', linewidth=2)       
plt.title('Distribution of Securities Share on 12/31/2022')
plt.show()


In [ ]:
# compute the correlation between FF Hike and Positive FF Shock:
df[['FF Hike', 'Positive FF Shock']].corr()

In [ ]:
df = compute_growth(df, transformations, window=1)

In [ ]:
print(df['Loan_Growth'].describe())
print(df['Loan_Growth'].quantile([0.01, 0.99]))

In [ ]:
print(df['Deposit_Growth'].describe())
print(df['Deposit_Growth'].quantile([0.01, 0.99]))

In [ ]:
# mask the outliers (top and bottom 0.1%) of the distribution of 'Deposit Growth':
df['Deposit_Growth'] = df['Deposit_Growth'].mask((df['Deposit_Growth'] < df['Deposit_Growth'].quantile(0.01)) | 
                                                 (df['Deposit_Growth'] > df['Deposit_Growth'].quantile(0.99)), np.nan)

df['Loan_Growth'] = df['Loan_Growth'].mask((df['Loan_Growth'] < df['Loan_Growth'].quantile(0.01)) | 
                                                 (df['Loan_Growth'] > df['Loan_Growth'].quantile(0.99)), np.nan)


In [ ]:
df['Securities Growth'] = df.groupby('IDRSSD')['Securities AC'].pct_change(fill_method=None)
print(df['Securities Growth'].describe())
# if the 'Securities Growth' is in either top or bottom 1% of the distribution, set it to NaN:
df['Securities Growth'] = df['Securities Growth'].mask((df['Securities Growth'] < df['Securities Growth'].quantile(0.01)) | (df['Securities Growth'] > df['Securities Growth'].quantile(0.99)))
print(df['Securities Growth'].describe())

In [ ]:
df['Real Cash'] = df['Cash'] / df['CPI'] * 100
df['Cash Growth'] = df.groupby('IDRSSD')['Real Cash'].pct_change(fill_method=None)
print(df['Cash Growth'].describe())
# if the 'Securities Growth' is in either top or bottom 1% of the distribution, set it to NaN:
df['Cash Growth'] = df['Cash Growth'].mask((df['Cash Growth'] < df['Cash Growth'].quantile(0.01)) | (df['Cash Growth'] > df['Cash Growth'].quantile(0.99)))
print(df['Cash Growth'].describe())

In [ ]:
def cross_sectional_plots(  df,                     # Dataframe with bank-level data 
                            criteria,               # Criteria to define the interest rate shock
                            num_quantiles=100):     # Number of quantiles for the binned scatter plot

    # Define the interest rate criteria:
    # 1. 'ff_hike': FF Hike is a simple increase in the FFR.
    # 2. 'mp_shock': MP Shock is a positive shock to the FFR.
    if criteria == 'ff_hike':
        # Filter conditions for Expansion and Contraction
        expansion_condition = (
                              (df['Recession'] == 0) & 
                              (df['FF Hike'] == 1) &
                              (df['High Inflation'] == 1)
                              )
        contraction_condition = (
                                (df['Recession'] == 1) &
                                (df['FF Hike'] == 1) &
                                (df['High Inflation'] == 1)
                                )

    elif criteria == 'mp_shock':
        # Filter conditions for Expansion and Contraction
        expansion_condition = (
                            (df['Recession'] == 0) &
                            (df['Positive FF Shock'] == 1) &
                            (df['High Inflation'] == 1)
                            )
        contraction_condition = (
                            (df['Recession'] == 1) &
                            (df['Positive FF Shock'] == 1) & 
                            (df['High Inflation'] == 1)
                            )

    else:
        print('Criteria not found')
        return None
    
    # --------------------------------- Histograms --------------------------------- #

    # make a histogram of the de-trended loans when the inflation is high, FF hike and Boom vs. Recession:
    lin = np.linspace(-0.25, 0.25, 150)
    plt.figure(figsize=(12, 6))
    sns.histplot(df[expansion_condition]['Loan_Growth'], 
                label='Expansion', bins=lin, stat='density', alpha=0.6)
    plt.axvline(df[expansion_condition]['Loan_Growth'].mean(), linestyle='--', linewidth=2, color='blue')
    sns.histplot(df[contraction_condition]['Loan_Growth'], 
                label='Contraction', bins=lin, alpha=0.6, stat='density')
    plt.axvline(df[contraction_condition]['Loan_Growth'].mean(), linestyle='--', linewidth=2, color='orange')
    plt.title('Loan Growth (High Inflation, FF Hike)')
    plt.xlabel('Loan Growth')
    plt.ylabel('Density')
    plt.legend()
    # print the std deviation of both distributions in the (-0.2,8) coordinate:
    plt.text(0.1, 7, 'Mean Expansion:             ' + str(round(df[expansion_condition]['Loan_Growth'].mean(), 4)))
    plt.text(0.1, 6.5, 'Std. Expansion:             ' + str(round(df[expansion_condition]['Loan_Growth'].std(), 4)))
    plt.text(0.1, 6, 'Mean Contraction:           ' + str(round(df[contraction_condition]['Loan_Growth'].mean(), 4)))
    plt.text(0.1, 5.5, 'Std. Contraction:           ' + str(round(df[contraction_condition]['Loan_Growth'].std(), 4)))
    plt.grid(True, which='both', linestyle='--', lw=0.5, alpha=0.5, color='lightgrey')
    plt.xlim(-0.13, 0.22)
    plt.show()

    #-------------------------------------------------------------------------------------------------#

    lin = np.linspace(-0.25, 0.25, 150)
    plt.figure(figsize=(12, 6))
    sns.histplot(df[expansion_condition]['Deposit_Growth'], 
                label='Expansion', bins=lin, stat='density', alpha=0.6)
    plt.axvline(df[expansion_condition]['Deposit_Growth'].mean(), linestyle='--', linewidth=2, color='blue')
    sns.histplot(df[contraction_condition]['Deposit_Growth'], 
                label='Contraction', bins=lin, alpha=0.6, stat='density')
    plt.axvline(df[contraction_condition]['Deposit_Growth'].mean(), linestyle='--', linewidth=2, color='orange')
    plt.title('Deposits Growth (High Inflation, FF Hike)')
    plt.xlabel('Deposits Growth')
    plt.ylabel('Density')
    plt.legend()
    # print the std deviation of both distributions in the (-0.2,8) coordinate:
    plt.text(0.1, 7, 'Mean Expansion:             ' + str(round(df[expansion_condition]['Deposit_Growth'].mean(), 4)))
    plt.text(0.1, 6.5, 'Std. Expansion:             ' + str(round(df[expansion_condition]['Deposit_Growth'].std(), 4)))
    plt.text(0.1, 6, 'Mean Contraction:           ' + str(round(df[contraction_condition]['Deposit_Growth'].mean(), 4)))
    plt.text(0.1, 5.5, 'Std. Contraction:           ' + str(round(df[contraction_condition]['Deposit_Growth'].std(), 4)))
    plt.grid(True, which='both', linestyle='--', lw=0.5, alpha=0.5, color='lightgrey')
    plt.xlim(-0.13, 0.22)
    plt.show()

    #-------------------------------------------------------------------------------------------------#

    # make a histogram of the de-trended loans when the inflation is high, FF hike and Boom vs. Recession:
    plt.figure(figsize=(12, 6))
    sns.histplot(df[expansion_condition]['Securities Growth'], 
                label='Expansion', bins=lin, stat='density', alpha=0.6)
    plt.axvline(df[expansion_condition]['Securities Growth'].mean(), linestyle='--', linewidth=2, color='blue')
    sns.histplot(df[contraction_condition]['Securities Growth'], 
                label='Contraction', bins=lin, alpha=0.6, stat='density')
    plt.axvline(df[contraction_condition]['Securities Growth'].mean(), linestyle='--', linewidth=2, color='orange')
    plt.title('Securities Growth (High Inflation, FF Hike)')
    plt.xlabel('Securities Growth')
    plt.ylabel('Density')
    plt.legend()
    # print the std deviation of both distributions in the (-0.2,8) coordinate:
    plt.text(0.1, 7.5, 'Mean Expansion:             ' + str(round(df[expansion_condition]['Securities Growth'].mean(), 4)))
    plt.text(0.1, 6.5, 'Std. Expansion:             ' + str(round(df[expansion_condition]['Securities Growth'].std(), 4)))
    plt.text(0.1, 5.5, 'Mean Contraction:           ' + str(round(df[contraction_condition]['Securities Growth'].mean(), 4)))
    plt.text(0.1, 4.5, 'Std. Contraction:           ' + str(round(df[contraction_condition]['Securities Growth'].std(), 4)))
    plt.grid(True, which='both', linestyle='--', lw=0.5, alpha=0.5, color='lightgrey')
    plt.xlim(-0.25, 0.25)
    plt.show()

    #-------------------------------------------------------------------------------------------------#

    # make a histogram of the de-trended loans when the inflation is high, FF hike and Boom vs. Recession:
    plt.figure(figsize=(12, 6))
    lin = np.linspace(df['Cash Growth'].min(), 2, 150)
    sns.histplot(df[expansion_condition]['Cash Growth'], 
                label='Expansion', bins=lin, stat='density', alpha=0.6)
    plt.axvline(df[expansion_condition]['Cash Growth'].mean(), linestyle='--', linewidth=2, color='blue')
    sns.histplot(df[contraction_condition]['Cash Growth'], 
                label='Contraction', bins=lin, alpha=0.6, stat='density')
    plt.axvline(df[contraction_condition]['Cash Growth'].mean(), linestyle='--', linewidth=2, color='orange')
    plt.title('Cash Growth (High Inflation, FF Hike)')
    plt.xlabel('Cash Growth')
    plt.ylabel('Density')
    plt.legend()
    # print the std deviation of both distributions in the (-0.2,8) coordinate:
    # print the std deviation of both distributions in the (-0.2,8) coordinate:
    plt.text(1, 1.4, 'Mean Expansion:             ' + str(round(df[expansion_condition]['Cash Growth'].mean(), 4)))
    plt.text(1, 1.3, 'Std. Expansion:             ' + str(round(df[expansion_condition]['Cash Growth'].std(), 4)))
    plt.text(1, 1.2, 'Mean Contraction:           ' + str(round(df[contraction_condition]['Cash Growth'].mean(), 4)))
    plt.text(1, 1.1, 'Std. Contraction:           ' + str(round(df[contraction_condition]['Cash Growth'].std(), 4)))
    plt.grid(True, which='both', linestyle='--', lw=0.5, alpha=0.5, color='lightgrey')
    plt.show()


    # --------------------------------- Scatter Plots --------------------------------- #
    #-------------------------------------------------------------------------------------------------#

    # Create the figure
    plt.figure(figsize=(12, 6))

    # Binned scatter for expansion
    binned_scatter(
    df.loc[expansion_condition, 'Loan_Growth'], 
    df.loc[expansion_condition, 'Securities Growth'], 
    q=num_quantiles, 
    #color='blue', 
    label='Expansion'
    )

    # Binned scatter for contraction
    binned_scatter(
    df.loc[contraction_condition, 'Loan_Growth'], 
    df.loc[contraction_condition, 'Securities Growth'], 
    q=num_quantiles, 
    #color='orange', 
    label='Contraction'
    )

    # Plot settings
    plt.title('Binned Loan Growth vs. Securities Growth (Quantile Bins)')
    plt.ylabel('Loan Growth')
    plt.xlabel('Securities Growth')
    plt.ylim(-0.05, 0.12)
    plt.legend()
    plt.grid(True, which='both', linestyle='--', lw=0.5, alpha=0.5, color='lightgrey')
    plt.show()

    #-------------------------------------------------------------------------------------------------#
    # Create the figure
    plt.figure(figsize=(12, 6))

    # Binned scatter for expansion
    binned_scatter(
    df.loc[expansion_condition, 'Deposit_Growth'], 
    df.loc[expansion_condition, 'Loan_Growth'], 
    q=num_quantiles, 
    #color='blue', 
    label='Expansion'
    )

    # Binned scatter for contraction
    binned_scatter(
    df.loc[contraction_condition, 'Deposit_Growth'], 
    df.loc[contraction_condition, 'Loan_Growth'], 
    q=num_quantiles, 
    #color='orange', 
    label='Contraction'
    )

    # Plot settings
    plt.title('Binned Deposits Growth vs. Loan Growth (Quantile Bins)')
    plt.ylabel('Loan Growth')
    plt.xlabel('Deposits Growth')
    plt.ylim(-0.01, 0.05)
    plt.xlim(-0.1, 0.2)
    plt.legend()
    plt.grid(True, which='both', linestyle='--', lw=0.5, alpha=0.5, color='lightgrey')
    plt.show()

    #-------------------------------------------------------------------------------------------------#
    # Create the figure
    plt.figure(figsize=(12, 6))

    # Binned scatter for expansion
    binned_scatter(
    df.loc[expansion_condition, 'Deposit_Growth'], 
    df.loc[expansion_condition, 'Cash Growth'], 
    q=num_quantiles, 
    #color='blue', 
    label='Expansion'
    )

    # Binned scatter for contraction
    binned_scatter(
    df.loc[contraction_condition, 'Deposit_Growth'], 
    df.loc[contraction_condition, 'Cash Growth'], 
    q=num_quantiles, 
    #color='orange', 
    label='Contraction'
    )

    # Plot settings
    plt.title('Binned Deposits Growth vs. Cash Growth (Quantile Bins)')
    plt.ylabel('Cash Growth')
    plt.xlabel('Deposits Growth')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', lw=0.5, alpha=0.5, color='lightgrey')
    plt.show()

    #-------------------------------------------------------------------------------------------------#
    # Create the figure
    plt.figure(figsize=(12, 6))

    # Binned scatter for expansion
    binned_scatter(
    df.loc[expansion_condition, 'Deposit_Growth'], 
    df.loc[expansion_condition, 'Securities Growth'], 
    q=num_quantiles, 
    #color='blue', 
    label='Expansion'
    )

    # Binned scatter for contraction
    binned_scatter(
    df.loc[contraction_condition, 'Deposit_Growth'], 
    df.loc[contraction_condition, 'Securities Growth'], 
    q=num_quantiles, 
    #color='orange', 
    label='Contraction'
    )

    # Plot settings
    plt.title('Binned Deposits Growth vs. Securities Growth (Quantile Bins)')
    plt.ylabel('Securities Growth')
    plt.xlabel('Deposits Growth')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', lw=0.5, alpha=0.5, color='lightgrey')
    plt.show()

In [ ]:
cross_sectional_plots(df, 'ff_hike', num_quantiles=50)

In [ ]:
cross_sectional_plots(df, 'mp_shock', num_quantiles=50)